+++++++++++++++++++++++++++

This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]

+++++++++++++++++++++++++++

### This is a utility script used to authenticate the user with CalNet and store the resulting auth and refresh tokens to be used in other scripts.

In [2]:
import os, sys
from boxsdk.exception import BoxAPIException

In [3]:
# Import two classes from the boxsdk module - Client and OAuth2
from boxsdk import Client, OAuth2

# Define client ID, client secret, and developer token.
CLIENT_ID = None
CLIENT_SECRET = None
ACCESS_TOKEN = None

# Read app info from text file
with open('boxsdk.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    ACCESS_TOKEN = app_cfg.readline()

function to store the oauth2 refresh token in a local file. This can be modified to use a keychain or other as required.

In [4]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open('apptoken.cfg', 'w') as f:
        f.write(refresh_token.strip())

In [5]:
def store_client_config(client_id, client_secret, redirect_uri):
    
    """Callback for storing client configuration."""
    with open('boxsdk.cfg', 'w') as f:
        f.write(client_id.strip() + '\n')
        f.write(client_secret.strip()+ '\n')
        f.write(redirect_uri.strip()+ '\n')

The client id and client secret are defined in the Box application created for this notebook. Create the application at the Box Developers site: https://berkeley.app.box.com/developers/services/edit/

The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens which are then continually refreshed.

In [6]:
# Define client ID, client secret, and developer token.
#CLIENT_ID = 'ui31zjry6g8lwvkp0zl58qcyp02tsnw4'
#CLIENT_SECRET = 'CgUOz7mCADC1w57lF9XrsTjB2XSy8aM6'
REDIRECT_URI = 'https://berkeley.account.box.com/login'

store_client_config(CLIENT_ID, CLIENT_SECRET, REDIRECT_URI)

# for debugging
print ("CLIENT_ID: ", CLIENT_ID )
print ("CLIENT_SECRET: ", CLIENT_SECRET )
print ("REDIRECT_URI: ", REDIRECT_URI )

CLIENT_ID:  e0pdajs0rwy9vfp629aqctjxd10l7e6v

CLIENT_SECRET:  beuCb6T7qHfBDrsQFPOlJzqwPXQVTyPx

REDIRECT_URI:  https://berkeley.account.box.com/login


Perform autentication then create globus client. 
Place the url that is returned into a browser to authenticate with CalNet. CalNet will return a code in the resulting url. 

In [14]:
from boxsdk import OAuth2
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    store_tokens=store_tokens
)

auth_url, csrf_token = oauth.get_authorization_url(REDIRECT_URI.strip())

print ("auth_url", auth_url)
#print ("csrf_token", csrf_token)

auth_url https://app.box.com/api/oauth2/authorize?state=box_csrf_token_i1xEh6OzWYBv1iwU&response_type=code&client_id=e0pdajs0rwy9vfp629aqctjxd10l7e6v&redirect_uri=https%3A%2F%2Fberkeley.account.box.com%2Flogin
csrf_token box_csrf_token_i1xEh6OzWYBv1iwU


Place the code provided in the line below before executing this cell.

__After selecting the Authorise button on the Box login page the following page will have "code=" at the end of the url. Copy the provided code into the following cell instead of clicking the Continue button on that page__

In [19]:
access_token, refresh_token = oauth.authenticate('pa1E0NSqlBmdNHXrs9x2SW4l7qx4u8qU')
#access_token, refresh_token = oauth.authenticate('lgbyixPBGs7Nw0nK1oTzKDvURuGxVwUS')
store_tokens(access_token, refresh_token)

BoxOAuthException: 
Message: b'{"error":"unauthorized_client","error_description":"The grant type is unauthorized for this client_id"}'
Status: 400
URL: https://api.box.com/oauth2/token
Method: POST

Create a client to Box and verify that the authorization is valid.
Verify client by retrieving the name of the users root folder in Box.

In [20]:
from boxsdk import Client

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

BoxOAuthException: 
Message: b'{"error":"unauthorized_client","error_description":"The grant type is unauthorized for this client_id"}'
Status: 400
URL: https://api.box.com/oauth2/token
Method: POST